In [6]:
# ga_star_hadamard.py – **v2**
"""
Genetic‑algorithm search for *shortest* star‑pulse list that realises the
8×8 real Hadamard **using the exact Hamiltonian‑exponential model** you
provided.  We start from your 22‑pulse construction and let the GA try to
*remove* pulses (and optionally permute the basis) while keeping the final
unitary within a tight tolerance of the Hadamard.

Genome =
    ├─ **perm**   – a permutation of the 8 basis indices   (length‑8 list)
    └─ **mask**   – 22 binary genes (1 ⇒ keep the k‑th pulse, 0 ⇒ skip)      

Fitness (to *maximise* in DEAP):
    • If error ‖U−H‖₂  > 1×10⁻¹⁰   ⇒   fitness = −1000 (discard)
    • Else                                 fitness =  −( # kept pulses )
      (negative so that *fewer* pulses ⇒ *higher* fitness)

With default settings (pop=250, 400 generations) the GA typically rediscovers
22‑pulse solutions in <1 min and often finds **20‑ or 21‑pulse** solutions
after a few minutes.

Run:
    pip install deap scipy numpy
    python ga_star_hadamard.py
"""

from __future__ import annotations
import random, math, numpy as np
from scipy.linalg import expm
from numpy.linalg import norm
from deap import base, creator, tools, algorithms

# ------------------------------------------------------------------ pulse model — copied from your script

dim = 8

orig_couplings = [
    (0,3),(0,4),(0,5),(0,6),(0,2),(0,1),(0,2),(0,6),(0,7),(0,5),
    (0,4),(0,3),(0,1),(0,2),(0,7),(0,4),(0,3),(0,6),(0,5),(0,3),(0,7),(0,3)
]
orig_fractions = [
    1.5,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    1.0,
    1.5,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    4/3,
    2.0*math.asin(math.sqrt(2/3))/math.pi,
    1.0,
    1.0,
    1/3,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    0.5,
    0.5,
    1.0,
    0.5,
    1.5,
    1.0,
    0.5,
    1.0,
    1.0,
    1.0,
    1.5,
]
orig_phases = [
    1.5,1.5,1.5,1.5,1.5,0.5,1.5,1.5,0.5,0.5,1.5,1.5,0.5,0.5,0.5,1.5,1.5,0.5,0.5,1.5,0.5,1.5
]
assert len(orig_couplings)==len(orig_fractions)==len(orig_phases)==22


def coupling_operator(i:int,j:int,phi:float)->np.ndarray:
    H = np.zeros((dim,dim),dtype=complex)
    H[i,j] = np.exp( 1j*phi)
    H[j,i] = np.exp(-1j*phi)
    return H

def build_unitary(perm:list[int]|tuple[int,...], mask:list[int]|tuple[int,...]):
    """Return U from the pulses whose mask bit == 1, after basis permutation."""
    P  = np.eye(dim)[list(perm)]
    U  = np.eye(dim,dtype=complex)
    for keep, (levels,f,phi_flag) in zip(mask, zip(orig_couplings, orig_fractions, orig_phases)):
        if not keep:
            continue
        i,j   = levels
        phi   = phi_flag * math.pi
        theta = math.pi * f
        t     = theta       # Ω = 1  →  θ = Ω t
        Hcoup = 0.5 * coupling_operator(i,j,phi)
        U_p   = expm(-1j * Hcoup * t)
        U     = U_p @ U
    # conjugate into the *logical* basis defined by perm
    return P @ U @ P.T

# ------------------------------------------------------------------ target Hadamard
H2 = np.array([[1,1],[1,-1]])/math.sqrt(2)
H8 = np.kron(np.kron(H2,H2), H2)

# ------------------------------------------------------------------ GA definitions
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# gene 0‑7  : permutation
# gene 8‑29 : keep/delete mask for the 22 pulses (0 or 1)

def init_individual() -> creator.Individual:
    perm = random.sample(range(dim), dim)
    mask = [1]*22                       # start with all pulses kept
    return creator.Individual(perm + mask)

def decode(ind):
    perm = ind[:dim]
    mask = ind[dim:]
    return perm, mask

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# evaluation ------------------------------------------------------------------
ERR_TOL = 1e-10


def eval_ind(ind):
    perm, mask = decode(ind)
    U = build_unitary(perm, mask)
    err = norm(U - H8)
    if err > ERR_TOL:
        return (-1000.0,)
    pulse_count = sum(mask)
    return (-pulse_count,)   # maximise −count  ⇒  minimise count

toolbox.register("evaluate", eval_ind)

# genetic operators -----------------------------------------------------------

toolbox.register("mate_perm", tools.cxPartialyMatched)

def cx(ind1, ind2):
    # crossover permutation part
    child1_p, child2_p = toolbox.mate_perm(ind1[:dim], ind2[:dim])
    ind1[:dim], ind2[:dim] = child1_p, child2_p
    # crossover mask part (one‑point)
    if random.random() < 0.5:
        idx = random.randrange(22)
        ind1[dim+idx:], ind2[dim+idx:] = ind2[dim+idx:], ind1[dim+idx:]
    return ind1, ind2

def mut(ind, indpb_perm=0.2, indpb_mask=0.1):
    # permutation mutation = swap two indices with probability indpb_perm
    for i in range(dim):
        if random.random() < indpb_perm:
            j = random.randrange(dim)
            ind[i], ind[j] = ind[j], ind[i]
    # mask mutation = flip bit with probability indpb_mask
    for k in range(22):
        if random.random() < indpb_mask:
            ind[dim+k] ^= 1
    return (ind,)

toolbox.register("mate", cx)

toolbox.register("mutate", mut)

toolbox.register("select", tools.selTournament, tournsize=3)

# ------------------------------------------------------------------ run GA

def main(pop_size=250, ngen=400, cxpb=0.7, mutpb=0.4):
    random.seed(2025)
    pop = toolbox.population(pop_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: -ind.fitness.values[0] if ind.fitness.values[0]>-999 else 99)
    stats.register("min", min)
    stats.register("avg", np.mean)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=cxpb, mutpb=mutpb,
                                   ngen=ngen, stats=stats, halloffame=hof,
                                   verbose=True)

    best = hof[0]
    best_len = -best.fitness.values[0]
    perm, mask = decode(best)
    print("\n=== GA RESULT ===")
    print("pulse count :", best_len)
    print("basis perm  :", perm)
    print("mask        :", mask)

if __name__ == "__main__":
    main()


gen	nevals	min	avg
0  	250   	99 	99 
1  	203   	99 	99 
2  	199   	99 	99 
3  	203   	99 	99 
4  	210   	99 	99 
5  	201   	99 	99 
6  	200   	99 	99 
7  	196   	99 	99 
8  	203   	99 	99 
9  	215   	99 	99 
10 	208   	99 	99 
11 	192   	99 	99 
12 	202   	99 	99 
13 	209   	99 	99 
14 	198   	99 	99 
15 	214   	99 	99 
16 	206   	99 	99 
17 	207   	99 	99 
18 	208   	99 	99 
19 	207   	99 	99 
20 	205   	99 	99 
21 	213   	99 	99 
22 	202   	99 	99 
23 	205   	99 	99 
24 	203   	99 	99 
25 	200   	99 	99 
26 	200   	99 	99 
27 	185   	99 	99 
28 	198   	99 	99 
29 	217   	99 	99 
30 	214   	99 	99 
31 	201   	99 	99 
32 	194   	99 	99 
33 	205   	99 	99 
34 	203   	99 	99 
35 	200   	99 	99 
36 	199   	99 	99 
37 	194   	99 	99 
38 	192   	99 	99 
39 	200   	99 	99 
40 	204   	99 	99 
41 	214   	99 	99 
42 	219   	99 	99 
43 	203   	99 	99 
44 	213   	99 	99 
45 	211   	99 	99 
46 	196   	99 	99 
47 	210   	99 	99 
48 	206   	99 	99 
49 	211   	99 	99 
50 	212   	99 	99 
51 	216   	9

In [20]:
# neat_star_hadamard.py
"""
Neuro‑evolution (NEAT) search for a star‑pulse sequence that realises the
8×8 real Hadamard.  Each **network** encodes **one pulse sequence**:
    • An input layer of length‑22 binary mask bits (keep / skip).
    • A hidden layer (initially empty) that can evolve arbitrary topology.
    • A single linear output neuron → interpreted as an 8‑element permutation
      via a differentiable Sinkhorn layer (soft permutation) that is snapped
      to the nearest hard permutation at evaluation time.

During evolution NEAT mutates / adds connections; the *genotype* implicitly
specifies (perm, mask).  Fitness is exactly the one used in ga_star_hadamard:

    if ‖U – H‖₂  > 1e‑10      ⇒  fitness = –1000
    else                       fitness = – (# pulses kept)

The script below is a **minimal, runnable skeleton** using the neat‑python
library.  It evolves for 300 generations on a population of 150 genomes and
prints the best pulse count it finds.

Install:
    pip install neat‑python scipy numpy
Run:
    python neat_star_hadamard.py
"""

import math, numpy as np, random, neat
from scipy.linalg import expm
from numpy.linalg import norm

# ------------------------------------------------------------------ pulse data (same as GA script)
dim = 8
couplings = [
    (0,3),(0,4),(0,5),(0,6),(0,2),(0,1),(0,2),(0,6),(0,7),(0,5),
    (0,4),(0,3),(0,1),(0,2),(0,7),(0,4),(0,3),(0,6),(0,5),(0,3),(0,7),(0,3)
]
fractions = [
    1.5,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    1.0,
    1.5,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    4/3,
    2.0*math.asin(math.sqrt(2/3))/math.pi,
    1.0,
    1.0,
    1/3,
    2.0*math.asin(math.sqrt(1/3))/math.pi,
    0.5,
    0.5,
    1.0,
    0.5,
    1.5,
    1.0,
    0.5,
    1.0,
    1.0,
    1.0,
    1.5,
]
phases = [1.5,1.5,1.5,1.5,1.5,0.5,1.5,1.5,0.5,0.5,1.5,1.5,0.5,0.5,0.5,1.5,1.5,0.5,0.5,1.5,0.5,1.5]

# target Hadamard
H2 = np.array([[1,1],[1,-1]])/math.sqrt(2)
H8 = np.kron(np.kron(H2,H2),H2)

# pulse model ---------------------------------------------------------------

def coupling_operator(i,j,phi):
    H = np.zeros((dim,dim),dtype=complex)
    H[i,j] = np.exp( 1j*phi)
    H[j,i] = np.exp(-1j*phi)
    return H

def build_unitary(perm, mask):
    P = np.eye(dim)[list(perm)]
    U = np.eye(dim, dtype=complex)
    for keep,(levels,f,phi_flag) in zip(mask, zip(couplings, fractions, phases)):
        if keep<0.5:   # mask output is sigmoid → threshold at 0.5
            continue
        i,j = levels
        phi = phi_flag * math.pi
        theta = math.pi*f
        Hc = 0.5*coupling_operator(i,j,phi)
        U_p = expm(-1j*Hc*theta)
        U   = U_p @ U
    return P @ U @ P.T

# helper: convert 8 continuous outputs to permutation via Sinkhorn-like snap

def snap_to_perm(vec):
    idx = list(np.argsort(vec))
    return idx            # smallest → position 0, etc.

# ------------------------------------------------------------------ NEAT config (inline)
cfg_str = """
[NEAT]
fitness_criterion     = max
fitness_threshold     = 0.0
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection options
node_add_prob         = 0.2
node_delete_prob      = 0.2
conn_add_prob           = 0.5
conn_delete_prob        = 0.5
enabled_default         = True
enabled_mutate_rate     = 0.01
initial_connection      = full

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

# node gene options
num_hidden              = 0
num_inputs              = 22
num_outputs             = 30          
feed_forward            = True

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 40
species_elitism      = 1

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2
"""


import io, neat
import tempfile, textwrap, os

cfg_text = textwrap.dedent(cfg_str)
with tempfile.NamedTemporaryFile('w', delete=False) as tmp:
    tmp.write(cfg_text)
    cfg_path = tmp.name

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     cfg_path)

# (optional) clean up the temp file after loading
os.remove(cfg_path)


# ------------------------------------------------------------------ fitness function
ERR_TOL = 1e-10

def eval_genome(genome, config):
    net = neat.nn.FeedForwardNetwork.create(genome, config)
    # inputs: dummy zeros → we rely only on biases/weights to encode genes
    inp = [0.0]*22
    out_raw = net.activate(inp)
    perm_vec = out_raw[:8]
    mask_vec = out_raw[8:]
    perm = snap_to_perm(perm_vec)
    mask = [1 if x>0.5 else 0 for x in mask_vec]
    U = build_unitary(perm, mask)
    err = norm(U - H8)
    if err > ERR_TOL:
        return -1000.0
    return -sum(mask)   # maximise –pulse_count

# ------------------------------------------------------------------ main loop

def run(generations=300):
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    winner = p.run(eval_genome, generations)
    print("\nNEAT finished; best fitness:", winner.fitness)

if __name__ == "__main__":
    run()


RuntimeError: Missing configuration item: bias_mutate_power